# k平均法によるクラスタリング

アルゴリズムの実装に必要なライブラリをインポートします。

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import randint
from tensorflow.keras.datasets import mnist

分類対象の画像データを準備します。ここでは、0, 3, 6 の3種類の数字データを全部で3000枚用意します。

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
chars = (0, 3, 6)
images = []
for image, label in zip(x_train.reshape([-1,784]), y_train):
  if label not in chars:
    continue
  images.append(image.tolist())
  if len(images) > 3000:
    break
images = np.array(images)

k 平均法によるクラスタリングを実施する関数を用意します。ここでは、代表点の更新を最大50回行います。誤差の変化が誤差の大きさの0.1%未満になった場合は、その時点で処理を打ち切ります。

In [0]:
def run_kmeans(images, k):
  cls = [0] * len(images)
  center = []
  for i in range(k):
    center.append(np.array([randint(256) for _ in range(784)]))

  err = 0.0
  for iter_num in range(50): 
    center_new = []
    for i in range(k):
      center_new.append(np.array([0 for _ in range(784)]))
    num_points = [0] * k
    err_new = 0.0

    for image, point in enumerate(images):
      min_dist = 256*256*784
      point = np.array(point)
      for i in range(k):
        d = sum([x*x for x in point-center[i]])
        if d < min_dist:
          min_dist = d
          cls[image] = i
        center_new[cls[image]] += point
        num_points[cls[image]] += 1
        err_new += 0.5 * min_dist

    for i in range(k):
      if num_points[i] == 0:
        center_new[i] = np.array([0 for _ in range(784)])
      else:
        center_new[i] = center_new[i] / num_points[i]
        
    center = center_new
    print ('E={}'.format(err_new))
    if iter_num > 0 and err - err_new < err * 0.001:
      break
    err = err_new
        
  print ('Done.')
  return center, cls

分類結果を画像イメージとして表示する関数を用意します。

In [0]:
def show_figure(mu, cls):
  K = len(mu)
  num = 6

  fig = plt.figure(figsize=(10, 4))
  for k in range(K):
    subplot = fig.add_subplot(K,num+1,1+k*(num+1))
    subplot.set_xticks([])
    subplot.set_yticks([])
    subplot.imshow(mu[k].reshape(28, 28), cmap=plt.cm.gray_r)
      
    i = 1
    for j in range(len(cls)):
      if cls[j] == k:
        subplot = fig.add_subplot(K,num+1,1+k*(num+1)+i)
        subplot.set_xticks([])
        subplot.set_yticks([])
        subplot.imshow(images[j].reshape(28,28), cmap=plt.cm.gray_r)
        i += 1
        if i > num:
          break

さきほど用意した関数を用いて、3つのクラスターに分類します。

In [5]:
result, cls = run_kmeans(images, 3)

E=70772853455.0
E=16040494901.534388
E=15528878612.538033
E=15410015851.491337
E=15341961304.957413
E=15275894780.044064
E=15217421111.199764
E=15178491106.62245
E=15157211108.525402
E=15150846087.451822
Done.


分類結果を画像イメージとして表示します。

In [6]:
show_figure(result, cls)